
# Job Shop Scheduling Problem 



## System Description (DFJSP)

The **Dynamic Flexible Job Shop Scheduling Problem (DFJSP)** schedules dynamically arriving jobs on multiple machines to minimize total tardiness.

- There are $n$ successively arriving jobs $J=\{J_1,\dots,J_n\}$ to be processed with $m$ machines $M=\{M_1,\dots,M_m\}$.

- Each job $J_i$ has $n_i$ ordered operations $ O_{i,1},\; O_{i,2},\; \dots,\; O_{i,n_i}$.
- Each operation $O_{i,j}$ can be processed on any compatible machine from its set $M_{i,j}\subseteq M$, with processing time $t_{i,j,k}$ on machine $M_k$.

- Each job has:
    - Arrival time $A_i$
    - Due date $D_i$
    - Operation completion time $C_{i,j}$

- To simplify the problem there are some constraints on the system.
    - **Capacity:** each machine processes at most one operation at a time.  
    - **Precedence:** operations of the same job follow the fixed order $O_{i,1}\!\to\! \cdots \!\to\! O_{i,n_i}$.  
    - **Non-preemption:** once started, an operation runs to completion.  
    - **No setups/transport:** setup and transfer times are neglected.

### Decisions
1. **Machine assignment (routing):** choose $M_k \in M_{i,j}$ for each $O_{i,j}$.
2. **Sequencing:** order operations waiting on each machine.

### Objective
Minimize total tardiness:
$$
\min \sum_{i=1}^{n} \max\big(C_{i,n_i} - D_i,\; 0\big).
$$

### Constraints (informal)
1. **Capacity:** each machine processes at most one operation at a time.  
2. **Precedence:** operations of the same job follow the fixed order $O_{i,1}\!\to\! \cdots \!\to\! O_{i,n_i}$.  
3. **Non-preemption:** once started, an operation runs to completion.  
4. **No setups/transport:** setup and transfer times are neglected.




## State Representaion:

-  In previous RL-based scheduling methods, state feature was defind as a some indicatorus of the production status, i.e. number of machines/jobs/operations in shop floor, the remaining processing time of uncompleted jobs, the current workload/queue length of each machine and so on. However, the problem of this approach is that in real world the number of jobs/machine/operations are large and can vary in a wide range, and taking these indicators as state decrease the generalizability of the RL agent. Since it can only perform well under the same same state size.
- To seprate the state representaion from the the direct indicator mentiond earlier, we use seven elaborately state feature with each value in the range of [0, 1].

At each rescheduling point *t*, the environment state is represented by the following features:

1. **Average machine utilization:**  $U_{ave}(t) = \frac{\sum_{k=1}^{m} U_k(t)}{m}$

2. **Standard deviation of machine utilization:**  $U_{std}(t) = \sqrt{\frac{\sum_{k=1}^{m} (U_k(t) - U_{ave}(t))^2}{m}}$

3. **Average operation completion rate:**  $CRO_{ave}(t) = \frac{\sum_{i=1}^{n} OP_i(t)}{\sum_{i=1}^{n} n_i}$

4. **Average job completion rate:**  $CRJ_{ave}(t) = \frac{\sum_{i=1}^{n} CRJ_i(t)}{n}$

5. **Standard deviation of job completion rate:**  $CRJ_{std}(t) = \sqrt{\frac{\sum_{i=1}^{n} (CRJ_i(t) - CRJ_{ave}(t))^2}{n}}$

6. **Estimated tardiness rate:**  $Tard_{e}(t) = \frac{N_{tard}}{N_{left}}$

7. **Actual tardiness rate:**  $Tard_{a}(t) = \frac{\text{Number of actual tardy operations}}{\text{Number of uncompleted operations}}$


## Action Space:

Before going explaning action space for the RL agent we need to explain what are the decision that need to be taken for succesful and efficient job shop scheduling.
There are two main categories of decision that need to be taken, **sequencing** and **machine assignment (routing)**.

- Sequencing: Determines the order in which jobs are processed on each machine.
- Machine assignment: Determines which machine will execute a specific operation when multiple machines are capable of performing it.


Traditionaly some rules have been used for job shop scheduling problem, but no specific rule has found to perform well across all shop configuration. Here the goal of reinforcemnet learning agent is to select between six different comosite of sequencing and machine assignment rule. In this way the reinforcement learning can learn to dispatch each rules based on the status of the system. 

List of the rules: 
| **Rule** | **Description** | **Formula / Logic** |
|-----------|-----------------|----------------------|
| **LWKRSPT** | Least Work Remaining, tie-break by Shortest Processing Time | Lexicographic rule: minimize **LWKR**, then **PT** |
| **LWKRMOD** | Least Work Remaining, tie-break by Modified Operation Due Date | Lexicographic rule: minimize **LWKR**, then **MOD** |
| **PTWINQ** | Processing Time plus Work In Next Queue | **Priority = PT + WINQ** |
| **PTWINQS** | Processing Time plus Work In Next Queue plus Slack | **Priority = PT + WINQ + Slack** |
| **DPTLWKRS** | Double Processing Time plus Least Work Remaining plus Slack | **Priority = 2×PT + LWKR + Slack** |
| **DPTWINQNPT** | Double Processing Time plus Work In Next Queue plus Next Processing Time | **Priority = 2×PT + WINQ + NPT** |

### Rule 1: **LWKRSPT**

In [ ]:
# This is a placeholder for the project code.

### Rule 2: **LWKRMOD**

In [ ]:
# This is a placeholder for the project code.

### Rule 3: **PTWINQ**

In [ ]:
# This is a placeholder for the project code.

### Rule 4: **PTWINQS**

In [ ]:
# This is a placeholder for the project code.

### Rule 5: **DPTLWKRS**

In [ ]:
# This is a placeholder for the project code.

### Rule 6: **DPTWINQNPT**

In [ ]:
# This is a placeholder for the project code.

## Reward definition:

The goal of the reward function is to lower these values follwing the same priority order.
1. Lower **actual tardiness**
2. Lower **estimated tardiness**
3. Higher **machine utilization**


### **How It Works**

#### **1. Primary goal — Minimize actual tardiness**
- If the **actual tardiness rate (Tarda)** decreases → **reward = +1**  
- If it increases → **reward = –1**

#### **2. Secondary goal — Reduce estimated tardiness**
- If the actual tardiness remains unchanged, compare the **estimated tardiness rate (Tarde):**  
  - If Tarde decreases → **reward = +1**  
  - If Tarde increases → **reward = –1**

#### **3. Tertiary goal — Maintain high machine utilization**
- If tardiness values remain unchanged, check the **average machine utilization (Uave):**  
  - If Uave increases → **reward = +1**  
  - If Uave remains within 95% of its previous value → **reward = 0**  
  - Otherwise → **reward = –1**




In [2]:
# Reward function implementation goes here.